# Social Science Quick Start: Social Network Analysis

**Duration:** 30-45 minutes  
**Goal:** Analyze Reddit social network to understand community structure and information diffusion

## What You'll Learn

- Load and explore a real social network graph (Reddit comments)
- Detect communities using graph algorithms
- Identify influential users using centrality measures
- Visualize network structure and dynamics
- Understand information diffusion patterns

## Dataset

We'll use a **Reddit Hyperlink Network** dataset:
- Nodes: Subreddits (online communities)
- Edges: Hyperlinks between subreddits
- Attributes: Subreddit properties, post activity
- Source: Stanford Large Network Dataset Collection (SNAP)

No AWS account or API keys needed - let's get started!

## 1. Setup and Data Loading

In [ ]:
# Import libraries (all pre-installed in Colab/Studio Lab)
import warnings
from collections import Counter
from datetime import datetime

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 10

print("✓ Libraries loaded successfully!")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d')}")

In [ ]:
# Download Reddit hyperlink network from SNAP
# This is a curated dataset of subreddit interactions
import os
import urllib.request

# URLs for the dataset
edges_url = "http://snap.stanford.edu/data/soc-redditHyperlinks-body.tsv"
node_props_url = "http://snap.stanford.edu/data/web-redditEmbeddings-subreddits.csv"

print("Downloading Reddit network data...")
print("This may take 2-3 minutes (~50MB)")

# Download edge list
edges_file = "reddit_edges.tsv"
if not os.path.exists(edges_file):
    urllib.request.urlretrieve(edges_url, edges_file)
    print(f"✓ Downloaded edge list: {edges_file}")
else:
    print(f"✓ Using cached edge list: {edges_file}")

# Load the network data
edges_df = pd.read_csv(edges_file, sep="\t")
print(f"\n✓ Loaded {len(edges_df):,} hyperlinks between subreddits")
edges_df.head()

### Understanding the Network

Each row represents a **hyperlink** from one subreddit to another:
- **SOURCE_SUBREDDIT:** The subreddit posting the link
- **TARGET_SUBREDDIT:** The subreddit being linked to
- **POST_ID:** The post containing the link
- **TIMESTAMP:** When the link was posted
- **LINK_SENTIMENT:** Positive or negative sentiment (1 or -1)

This creates a **directed network** where edges show information flow between communities.

## 2. Network Construction

In [ ]:
# Build directed network graph
print("Building network graph...")

G = nx.DiGraph()

# Add edges with properties
for _, row in edges_df.iterrows():
    source = row["SOURCE_SUBREDDIT"]
    target = row["TARGET_SUBREDDIT"]
    sentiment = row["LINK_SENTIMENT"]

    if G.has_edge(source, target):
        # Increment weight if edge exists
        G[source][target]["weight"] += 1
    else:
        # Add new edge
        G.add_edge(source, target, weight=1, sentiment=sentiment)

print("\n=== Network Statistics ===")
print(f"Nodes (subreddits): {G.number_of_nodes():,}")
print(f"Edges (hyperlinks): {G.number_of_edges():,}")
print(f"Network density: {nx.density(G):.6f}")
print(f"Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")

In [ ]:
# Calculate basic network properties
print("\n=== Network Connectivity ===")

# Check if network is strongly connected
is_strongly_connected = nx.is_strongly_connected(G)
print(f"Strongly connected: {is_strongly_connected}")

# Find largest strongly connected component
largest_scc = max(nx.strongly_connected_components(G), key=len)
print(
    f"Largest strongly connected component: {len(largest_scc):,} nodes ({len(largest_scc) / G.number_of_nodes() * 100:.1f}%)"
)

# Find largest weakly connected component
largest_wcc = max(nx.weakly_connected_components(G), key=len)
print(
    f"Largest weakly connected component: {len(largest_wcc):,} nodes ({len(largest_wcc) / G.number_of_nodes() * 100:.1f}%)"
)

# Work with largest component for analysis
G_main = G.subgraph(largest_wcc).copy()
print(f"\n✓ Using main component with {G_main.number_of_nodes():,} nodes for analysis")

## 3. Degree Distribution Analysis

In [ ]:
# Calculate degree statistics
in_degrees = dict(G_main.in_degree())
out_degrees = dict(G_main.out_degree())

in_degree_values = list(in_degrees.values())
out_degree_values = list(out_degrees.values())

print("=== Degree Statistics ===")
print("\nIn-degree (links received):")
print(f"  Mean: {np.mean(in_degree_values):.2f}")
print(f"  Median: {np.median(in_degree_values):.2f}")
print(f"  Max: {np.max(in_degree_values)}")

print("\nOut-degree (links posted):")
print(f"  Mean: {np.mean(out_degree_values):.2f}")
print(f"  Median: {np.median(out_degree_values):.2f}")
print(f"  Max: {np.max(out_degree_values)}")

In [ ]:
# Visualize degree distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# In-degree distribution (log scale)
in_degree_counts = Counter(in_degree_values)
degrees, counts = zip(*sorted(in_degree_counts.items()))
axes[0].loglog(degrees, counts, "bo-", alpha=0.6, markersize=4)
axes[0].set_xlabel("In-Degree (links received)", fontsize=12, fontweight="bold")
axes[0].set_ylabel("Number of Subreddits", fontsize=12, fontweight="bold")
axes[0].set_title("In-Degree Distribution (Log-Log)", fontsize=13, fontweight="bold")
axes[0].grid(True, alpha=0.3)

# Out-degree distribution (log scale)
out_degree_counts = Counter(out_degree_values)
degrees, counts = zip(*sorted(out_degree_counts.items()))
axes[1].loglog(degrees, counts, "ro-", alpha=0.6, markersize=4)
axes[1].set_xlabel("Out-Degree (links posted)", fontsize=12, fontweight="bold")
axes[1].set_ylabel("Number of Subreddits", fontsize=12, fontweight="bold")
axes[1].set_title("Out-Degree Distribution (Log-Log)", fontsize=13, fontweight="bold")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Log-log plot shows power-law distribution - characteristic of scale-free networks")
print("   Most subreddits have few connections, but some 'hubs' have many connections")

## 4. Influence Analysis - Centrality Measures

In [ ]:
# Calculate various centrality measures
print("Calculating centrality measures...")
print("This may take 2-3 minutes for large networks...\n")

# PageRank - measures importance based on incoming links
pagerank = nx.pagerank(G_main, alpha=0.85)

# Betweenness centrality - measures bridging between communities
# Use sampling for large networks to speed up computation
betweenness = nx.betweenness_centrality(G_main, k=min(1000, G_main.number_of_nodes()))

# Degree centrality - simple measure based on number of connections
in_degree_centrality = nx.in_degree_centrality(G_main)
out_degree_centrality = nx.out_degree_centrality(G_main)

print("✓ Centrality measures calculated")

In [ ]:
# Find top influential subreddits by different metrics
print("=== Top 10 Most Influential Subreddits ===")

print("\n📍 By PageRank (overall importance):")
top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]
for i, (subreddit, score) in enumerate(top_pagerank, 1):
    print(f"  {i}. r/{subreddit}: {score:.6f}")

print("\n🌉 By Betweenness Centrality (bridge communities):")
top_betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)[:10]
for i, (subreddit, score) in enumerate(top_betweenness, 1):
    print(f"  {i}. r/{subreddit}: {score:.6f}")

print("\n📥 By In-Degree (most linked to):")
top_in_degree = sorted(in_degrees.items(), key=lambda x: x[1], reverse=True)[:10]
for i, (subreddit, degree) in enumerate(top_in_degree, 1):
    print(f"  {i}. r/{subreddit}: {degree} incoming links")

## 5. Community Detection

In [ ]:
# Convert to undirected for community detection
G_undirected = G_main.to_undirected()

print("Detecting communities using Louvain algorithm...")
print("This may take 1-2 minutes...\n")

# Use Louvain community detection
from networkx.algorithms import community

communities = community.greedy_modularity_communities(G_undirected)

print(f"✓ Detected {len(communities)} communities")
print("\n=== Community Sizes ===")

# Sort communities by size
sorted_communities = sorted(communities, key=len, reverse=True)

for i, comm in enumerate(sorted_communities[:10], 1):
    print(f"Community {i}: {len(comm):,} subreddits")

# Calculate modularity
modularity = community.modularity(G_undirected, communities)
print(f"\nModularity: {modularity:.4f}")
print("(Higher modularity = stronger community structure)")

In [ ]:
# Analyze largest communities
print("\n=== Sample Subreddits from Top Communities ===")

for i, comm in enumerate(sorted_communities[:5], 1):
    print(f"\nCommunity {i} ({len(comm)} subreddits):")
    # Show top 10 most influential nodes in this community
    comm_pagerank = {node: pagerank[node] for node in comm if node in pagerank}
    top_in_comm = sorted(comm_pagerank.items(), key=lambda x: x[1], reverse=True)[:10]
    print("  Top subreddits: " + ", ".join([f"r/{node}" for node, _ in top_in_comm]))

## 6. Network Visualization

In [ ]:
# Create visualization of network structure
# Use a subset for visualization (full network too large)
print("Creating network visualization...")

# Select top nodes by PageRank for visualization
top_nodes = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:100]
top_node_names = [node for node, _ in top_nodes]

# Create subgraph
G_viz = G_main.subgraph(top_node_names).copy()

# Assign communities to nodes
node_communities = {}
for i, comm in enumerate(sorted_communities):
    for node in comm:
        node_communities[node] = i

# Create color map
node_colors = [node_communities.get(node, -1) for node in G_viz.nodes()]
node_sizes = [pagerank[node] * 50000 for node in G_viz.nodes()]

# Draw network
plt.figure(figsize=(16, 12))
pos = nx.spring_layout(G_viz, k=0.5, iterations=50, seed=42)

nx.draw_networkx_nodes(
    G_viz,
    pos,
    node_color=node_colors,
    node_size=node_sizes,
    cmap="tab20",
    alpha=0.7,
    edgecolors="black",
    linewidths=0.5,
)

nx.draw_networkx_edges(
    G_viz, pos, alpha=0.2, arrows=True, arrowsize=10, edge_color="gray", width=0.5
)

# Label top 20 nodes
top_20_nodes = top_node_names[:20]
labels = {node: f"r/{node}" for node in top_20_nodes}
nx.draw_networkx_labels(G_viz, pos, labels, font_size=8, font_weight="bold")

plt.title(
    "Reddit Subreddit Network - Top 100 by PageRank\nNode size = influence, Color = community",
    fontsize=15,
    fontweight="bold",
    pad=20,
)
plt.axis("off")
plt.tight_layout()
plt.show()

print("\n📊 Visualization shows network structure with:")
print("   • Node size proportional to PageRank (influence)")
print("   • Node color represents detected community")
print("   • Arrows show direction of hyperlinks")

## 7. Information Diffusion Analysis

In [ ]:
# Analyze information spread potential
print("=== Information Diffusion Potential ===")

# Calculate clustering coefficient (how connected neighbors are)
clustering = nx.clustering(G_undirected)
avg_clustering = np.mean(list(clustering.values()))

print(f"\nAverage clustering coefficient: {avg_clustering:.4f}")
print("(Higher = more tightly clustered communities)")

# Calculate average shortest path length for sample
# Use largest connected component
largest_cc = max(nx.connected_components(G_undirected), key=len)
G_cc = G_undirected.subgraph(largest_cc)

# Sample nodes for path length calculation
sample_size = min(1000, len(G_cc.nodes()))
sample_nodes = np.random.choice(list(G_cc.nodes()), size=sample_size, replace=False)
G_sample = G_cc.subgraph(sample_nodes)

avg_path_length = nx.average_shortest_path_length(G_sample)
print(f"\nAverage shortest path length: {avg_path_length:.2f} hops")
print("(Information spreads this many steps on average)")

# Calculate diameter
diameter = nx.diameter(G_sample)
print(f"Network diameter: {diameter} hops")
print("(Maximum distance between any two subreddits)")

In [ ]:
# Identify key information spreaders
print("\n=== Top Information Spreaders ===")
print("(Subreddits that bridge multiple communities)\n")

# Combine metrics to identify super-spreaders
spreader_scores = {}
for node in G_main.nodes():
    # Weighted combination of metrics
    score = (
        0.4 * pagerank.get(node, 0) * 1000  # Influence
        + 0.3 * betweenness.get(node, 0) * 100  # Bridging
        + 0.3 * out_degree_centrality.get(node, 0) * 10  # Activity
    )
    spreader_scores[node] = score

top_spreaders = sorted(spreader_scores.items(), key=lambda x: x[1], reverse=True)[:15]

for i, (subreddit, score) in enumerate(top_spreaders, 1):
    in_deg = in_degrees.get(subreddit, 0)
    out_deg = out_degrees.get(subreddit, 0)
    pr = pagerank.get(subreddit, 0)
    print(
        f"{i:2d}. r/{subreddit:30s} | Score: {score:6.2f} | In: {in_deg:4d} | Out: {out_deg:4d} | PR: {pr:.5f}"
    )

## 8. Summary and Key Findings

In [ ]:
# Generate summary report
print("=" * 70)
print("SOCIAL NETWORK ANALYSIS SUMMARY")
print("=" * 70)

print("\n📊 NETWORK STRUCTURE:")
print(f"   • Total subreddits (nodes): {G_main.number_of_nodes():,}")
print(f"   • Total hyperlinks (edges): {G_main.number_of_edges():,}")
print(
    f"   • Average connections per subreddit: {sum(dict(G_main.degree()).values()) / G_main.number_of_nodes():.1f}"
)
print(f"   • Network density: {nx.density(G_main):.6f}")

print("\n🔍 COMMUNITY STRUCTURE:")
print(f"   • Detected communities: {len(communities)}")
print(f"   • Modularity score: {modularity:.4f}")
print(f"   • Largest community: {len(sorted_communities[0]):,} subreddits")
print(f"   • Average clustering: {avg_clustering:.4f}")

print("\n📈 INFLUENCE & DIFFUSION:")
print(f"   • Top influencer: r/{top_pagerank[0][0]} (PageRank: {top_pagerank[0][1]:.6f})")
print(f"   • Top bridge: r/{top_betweenness[0][0]} (Betweenness: {top_betweenness[0][1]:.6f})")
print(f"   • Average path length: {avg_path_length:.2f} hops")
print(f"   • Network diameter: {diameter} hops")

print("\n⚡ KEY INSIGHTS:")
print("   • Network exhibits scale-free properties (power-law degree distribution)")
print(f"   • Small-world effect: information spreads quickly (~{avg_path_length:.0f} hops average)")
print(f"   • Strong community structure detected (modularity = {modularity:.2f})")
print("   • Few highly influential 'hub' subreddits control information flow")

print("\n✅ CONCLUSION:")
print("   The Reddit network shows typical social network characteristics: scale-free")
print("   topology, strong community clustering, and efficient information diffusion.")
print("   Targeting top influencers could amplify message reach by orders of magnitude.")
print("=" * 70)

## What You Learned

In 30-45 minutes, you:

1. ✅ Loaded and analyzed a real-world social network (50K+ nodes)
2. ✅ Calculated influence metrics (PageRank, centrality measures)
3. ✅ Detected communities using graph algorithms
4. ✅ Visualized network structure and dynamics
5. ✅ Analyzed information diffusion patterns
6. ✅ Identified key influencers and information spreaders

## Next Steps

### Ready for More?

**Tier 1: SageMaker Studio Lab (4-8 hours, free)**
- Analyze multiple platforms (Twitter, Reddit, Facebook)
- Train Graph Neural Networks for influence prediction
- Temporal dynamics analysis (how networks evolve)
- Persistent storage for 10GB+ datasets

**Tier 2: AWS Starter (2-4 hours, $5-15)**
- Store graphs in Neptune (managed graph database)
- Real-time influence tracking
- Automated analysis with Lambda

**Tier 3: Production Infrastructure (4-5 days, $50-500/month)**
- Multi-platform integration (10+ social networks)
- Streaming data ingestion and analysis
- Distributed graph processing with Neptune
- AI-powered insights with Amazon Bedrock

## Learn More

- **Dataset:** [Stanford Network Analysis Project (SNAP)](http://snap.stanford.edu/data/)
- **NetworkX Documentation:** [networkx.org](https://networkx.org/)
- **Graph Neural Networks:** [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/)

---

**Generated with [Claude Code](https://claude.com/claude-code)**